## 01. Pemahaman Domain (Domain Understanding)

---



In [104]:
import pandas as pd
import re
import nltk
import ipykernel

In [105]:
data1 = pd.read_csv('crawling_tweet/timnas_1.csv')
data2 = pd.read_csv('crawling_tweet/timnas_2.csv')
data3 = pd.read_csv('crawling_tweet/timnas_3.csv')
data4 = pd.read_csv('crawling_tweet/timnas_4.csv')
data5 = pd.read_csv('crawling_tweet/timnas (6).csv')
data6 = pd.read_csv('crawling_tweet/timnas (7).csv')
data7 = pd.read_csv('crawling_tweet/timnas (8).csv')
data8 = pd.read_csv('crawling_tweet/timnas (9).csv')
data9 = pd.read_csv('crawling_tweet/timnas (10).csv')
data10 = pd.read_csv('crawling_tweet/timnas (11).csv')
data11 = pd.read_csv('crawling_tweet/timnas (12).csv')

In [106]:
data = pd.concat([data1, data5,data6,data7,data8,data9,data10,data11], ignore_index=True)

In [107]:
data 

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1800679330726715550,Tue Jun 11 23:59:42 +0000 2024,1,TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY,1800679330726715550,NaN,NaN,in,Indonesia,0,0,0,https://x.com/tigapuluhtigax/status/1800679330...,1188470356668895234,tigapuluhtigax
1,1800679305925800015,Tue Jun 11 23:59:36 +0000 2024,1,ngerasa ga sih sekarang timnas kalo main pakek...,1800679305925800015,NaN,NaN,in,"Jombang, Indonesia",0,0,0,https://x.com/sotongism/status/180067930592580...,961056397063409664,sotongism
2,1800679288326476196,Tue Jun 11 23:59:31 +0000 2024,0,2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...,1800679288326476196,NaN,NaN,in,NaN,0,0,0,https://x.com/peanutkacang123/status/180067928...,1464596330421100548,peanutkacang123
3,1800589420766355730,Tue Jun 11 23:59:28 +0000 2024,0,@womensfootie_id Tinggal tmbah pemain diaspora...,1800679273185047018,NaN,womensfootie_id,in,NaN,0,0,0,https://x.com/PKananku1927/status/180067927318...,1772880651248939008,PKananku1927
4,1800679270857286043,Tue Jun 11 23:59:27 +0000 2024,0,Timnas Indonesia Vs Timnas Filipina 2-0 Indone...,1800679270857286043,NaN,NaN,in,NaN,0,0,0,https://x.com/kangsil2012/status/1800679270857...,546890263,kangsil2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,1800665012903813409,Tue Jun 11 23:02:48 +0000 2024,0,Daftar Negara Lolos ke Putaran 3 Kualifikasi P...,1800665012903813409,NaN,NaN,in,"Balikpapan, Indonesia",0,0,0,https://x.com/tribunkaltim/status/180066501290...,103551225,tribunkaltim
3060,1800580031087014330,Tue Jun 11 17:25:07 +0000 2024,1,Anang Ngerusak Momen Haru #TimNasDay Emang dar...,1800580031087014330,NaN,NaN,in,Salatiga - Jakarta - London,0,0,0,https://x.com/ronitoxid/status/180058003108701...,39716466,ronitoxid
3061,1800634437505343804,Tue Jun 11 21:01:18 +0000 2024,1,STY blak-blakan mengatakan stadion harusnya le...,1800634437505343804,NaN,NaN,in,Jakarta Capital Region,1,1,1,https://x.com/suaradotcom/status/1800634437505...,2340155395,suaradotcom
3062,1800672173994262918,Tue Jun 11 23:31:15 +0000 2024,0,Momen Timnas berlaga di Stadion jangan sekali-...,1800672173994262918,NaN,NaN,in,NaN,0,0,0,https://x.com/macanutara_/status/1800672173994...,1135805814642335745,macanutara_


In [108]:
data = data.drop_duplicates(subset='tweet_url')


In [109]:
jumlah_baris = data.shape[0]
print(f'Jumlah baris: {jumlah_baris}')

Jumlah baris: 1914


In [110]:
data.to_csv('crawling_tweet/timnas.csv', index=False)

## 1. Preposessing

In [111]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


# Create stopword list
factory = StopWordRemoverFactory()
stop_words = set(factory.get_stop_words())

# Tambahkan kata hubung tambahan ke dalam daftar stopwords
kata_hubung = {'dan', 'atau', 'tetapi', 'namun', 'sehingga', 'karena', 'jika', 'jikalau', 'maka', 'sebab', 'meskipun', 'walaupun', 'yg','lu','gue','gini','klo','sama','amp','you','and'}
stop_words.update(kata_hubung)

# Function to preprocess tweets
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions
    tweet = re.sub(r'#\w+', '', tweet)  # Remove hashtags
    tweet = re.sub(r'\d+', '', tweet)  # Remove numbers
    tweet = tweet.lower()  # Convert to lowercase
    tweet = re.sub(r'[^\w\s]', '', tweet)  # Remove punctuation
    tweet = tweet.split()  # Split into words
    tweet = [word for word in tweet if word not in stop_words]  # Remove stopwords
    return ' '.join(tweet)

# Apply preprocessing to each tweet
data['cleaned_tweet'] = data['full_text'].apply(preprocess_tweet)


C:\Users\TEMP\AppData\Local\Temp\ipykernel_14868\4186273742.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_tweet'] = data['full_text'].apply(preprocess_tweet)


## 2. Topic Modeling dengan LDA

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the cleaned tweets
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=list(stop_words))
tweet_matrix = vectorizer.fit_transform(data['cleaned_tweet'])

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=64)
lda.fit(tweet_matrix)

# Display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]])
    return topics

num_top_words = 16
feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, num_top_words)

for idx, topic in enumerate(topics):
    print(f"Topik {idx+1}: {', '.join(topic)}")


c:\Users\TEMP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kurangnya', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


Topik 1: gbk, timnas, rumput, stadion, mas, indonesia, konser, haye, pemain, main, putih, banget, prof, gak, thom, indo
Topik 2: anang, timnas, lagu, nyanyi, indonesia, pusaka, pemain, pertandingan, tanah, top, gak, suporter, liat, airku, tengah, momen
Topik 3: timnas, indonesia, negara, cup, asean, satusatunya, world, tenggara, bgt, wakil, menang, the, round, lawan, pildun, teu
Topik 4: ernando, verdonk, filipina, anang, syifa, haye, jay, bang, nathan, adrian, thom, indonesia, paes, towel, alhamdulillah, lepas
Topik 5: timnas, nonton, kasih, menang, terima, banget, bang, main, ga, indonesia, semoga, gw, kalah, pas, bareng, indo
Topik 6: piala, indonesia, dunia, timnas, kualifikasi, lolos, asia, ketiga, putaran, filipina, babak, gol, haye, zona, garuda, sejarah
Topik 7: timnas, keren, menang, indonesia, pot, bahrain, bangga, lawan, gak, malam, ga, qatar, tim, bgt, full, senyum
Topik 8: timnas, aja, udah, ga, kalo, sih, indonesia, gak, main, bola, persib, pemain, nih, banget, mah, bikin

## 3. Clustering

In [113]:
import numpy as np

# Get topic distribution for each tweet
topic_distribution = lda.transform(tweet_matrix)

# Assign each tweet to the topic with the highest probability
data['topic_cluster'] = np.argmax(topic_distribution, axis=1)

# Save the DataFrame to a CSV file
data.to_csv('tweets_with_topics.csv', index=False)

# Display the first few rows with assigned topics
print(data[['full_text', 'topic_cluster']].head())


                                           full_text  topic_cluster
0  TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY              8
1  ngerasa ga sih sekarang timnas kalo main pakek...              7
2  2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...              9
3  @womensfootie_id Tinggal tmbah pemain diaspora...              7
4  Timnas Indonesia Vs Timnas Filipina 2-0 Indone...              5


C:\Users\TEMP\AppData\Local\Temp\ipykernel_14868\3514048034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['topic_cluster'] = np.argmax(topic_distribution, axis=1)


## 4. Interpretasi

In [114]:
data = data[['full_text', 'topic_cluster']]

data = data.sort_values(by='topic_cluster').reset_index(drop=True)

In [115]:
data.to_csv('tweet+topics.csv', index=False)

In [116]:
data

,full_text,topic_cluster
0,Miris! Thom Haye Gagal Selebrasi Knee Slide Ga...,0
1,@TimnasIndonesia cek gol-gol cantik indo booss...,0
2,@TimnasIndonesia menyala prof toha!,0
3,emang setiap pake baju putih tuh wangiiii #Tim...,0
4,Menteri Komunikasi dan Informatika Indonrsia B...,0
...,...,...
1909,@PelatihT1dur Itu di minta lgsg sm timnas nya ...,9
1910,@gilabola_ina Timnas indonesia ko sekarang som...,9
1911,pemain timnas udh bener di private room kenapa...,9
1912,Pelatih Filipina Tom Saintfiet menanti kabar r...,9


# Topik 1: Kondisi Rumput dan Stadion GBK
Banyak diskusi tentang kondisi rumput di Stadion Gelora Bung Karno (GBK) setelah konser. Para penggemar mengeluhkan dampaknya pada permainan timnas Indonesia, terutama terkait performa pemain di lapangan.

# Topik 2: Penampilan Anang Menyanyikan Lagu Kebangsaan
Anang sering disebut dalam konteks menyanyikan lagu "Indonesia Pusaka" sebelum pertandingan timnas Indonesia. Momen ini menciptakan suasana emosional bagi para suporter dan menjadi bagian penting dari pengalaman menonton pertandingan.

# Topik 3: Prestasi Timnas di Kualifikasi Piala Dunia dan Asia
Banyak pembicaraan tentang keberhasilan timnas Indonesia lolos ke putaran ketiga kualifikasi Piala Dunia 2026 dan Piala Asia. Penggemar merasa bangga dan membahas strategi serta persiapan untuk pertandingan selanjutnya.

# Topik 4: Kemenangan Melawan Filipina
Pertandingan melawan Filipina menjadi sorotan dengan banyak diskusi tentang performa pemain seperti Verdonk dan Haye. Kemenangan ini dianggap sebagai momen bersejarah dan menjadi topik hangat di media sosial.

# Topik 5: Pengalaman Menonton Pertandingan Timnas
Banyak penggemar yang berbagi pengalaman menonton pertandingan timnas Indonesia. Mereka membahas momen-momen penting, pembelian tiket, dan harapan untuk kemenangan timnas di masa depan. Rasa fomo (fear of missing out) juga sering dibicarakan.

# Topik 6: Dukungan Suporter dan Kebanggaan
Dukungan dari suporter untuk timnas Indonesia menjadi topik utama. Banyak yang mengungkapkan rasa bangga setelah kemenangan dalam berbagai pertandingan dan berbagi ucapan selamat kepada para pemain.

# Topik 7: Strategi dan Performa Pemain
Diskusi tentang strategi yang digunakan oleh pelatih Shin Tae-yong dan performa pemain seperti Ernando dan Verdonk sering muncul. Penggemar juga membicarakan potensi pemain diaspora untuk memperkuat timnas.

# Topik 8: Perjalanan di Piala Dunia dan Asia
Perjalanan timnas Indonesia di babak kualifikasi Piala Dunia dan Piala Asia menjadi topik yang sering dibahas. Banyak yang merasa bangga dengan pencapaian timnas dan membahas pertandingan-pertandingan yang akan datang.

# Topik 9: Momen Bersejarah dan Gol Penting
Banyak yang membicarakan momen-momen bersejarah dan gol-gol penting yang dicetak oleh timnas Indonesia. Kemenangan di babak kualifikasi dan turnamen AFF menjadi sorotan utama.

# Topik 10: Kondisi dan Performa di Lapangan
Kondisi lapangan dan performa pemain di berbagai pertandingan sering menjadi topik diskusi. Banyak yang mengeluhkan kondisi lapangan yang kurang baik dan dampaknya pada permainan timnas.